# LFT: Ontology of the Information Space $I$

**Position in sequence:** foundational pre‑theory (inserted after `00a_Foundations`).

## Goal
Give a **minimal, formal ontology** for the pre‑geometric information space $I$ in
$$A = L(I),$$
compatible with LFT’s derivations. We model $I$ as the **space of potential distinctions** encoded as **partial orders** on a countable carrier set. The logical operator $L$ (Identity, Non‑Contradiction, Excluded Middle) acts as a **filter** that (i) rejects contradictions, (ii) closes under identity/reflexive consistency, and (iii) completes feasible structures to **total orders** (actualities).

## 1) Working Ontology for $I$

**Carrier (labels):** a finite subset $X=\{1,\dots,N\}$ of a countable set.  
**Raw structure:** a directed graph $G=(X,E)$ encoding *proposed distinctions* ($i\prec j$ meaning “$i$ distinguished before $j$”).  
**$I$ as a space:** all finite digraphs on $X$ with edges interpreted as *candidate order relations* (not yet assumed acyclic/transitive).  

Logical filtering $L$ acts to produce **feasible posets** and (if possible) **total orders**:
- **ID:** ensure reflexive self‑consistency (we treat identity as implicit; graphs omit self‑loops).  
- **NC:** reject cycles (acyclicity).  
- **EM:** when a **linear extension** exists, complete to one such extension (a total order).  

This matches `00b–00d` and supports later notebooks (geometry/dynamics/qudit bridge).

## 2) Minimal Data Structures & $L$ Implementation
We provide small utilities to:
1. Represent candidate relations and test **acyclicity** (Non‑Contradiction).  
2. Compute **linear extensions** (Excluded Middle → completion).  
3. Measure **inversion count** for a total order (hook‑in to `03_Dynamics_Time`).

In [ ]:
from collections import defaultdict, deque
import numpy as np

def is_acyclic(N, edges):
    """Kahn's algorithm for cycle check on directed graph over nodes 1..N."""
    indeg = [0]*(N+1)
    adj = [[] for _ in range(N+1)]
    for u,v in edges:
        adj[u].append(v)
        indeg[v]+=1
    q = deque([i for i in range(1,N+1) if indeg[i]==0])
    seen = 0
    while q:
        u = q.popleft(); seen+=1
        for w in adj[u]:
            indeg[w]-=1
            if indeg[w]==0:
                q.append(w)
    return seen==N

def linear_extensions(N, edges, limit=None):
    """Enumerate linear extensions (total orders) of a DAG.
    For N<=8 this is feasible; 'limit' caps the count for speed.
    Returns list of permutations (tuples) or a truncated list if limit set.
    """
    # Build adjacency & indegrees
    adj = [[] for _ in range(N+1)]
    indeg = [0]*(N+1)
    for u,v in edges:
        adj[u].append(v)
        indeg[v]+=1
    avail = [i for i in range(1,N+1) if indeg[i]==0]
    out = []
    stack = [(tuple(), tuple(sorted(avail)), tuple(indeg))]
    visited = 0
    while stack:
        pref, avail, indeg_t = stack.pop()
        if len(pref)==N:
            out.append(pref)
            if limit and len(out)>=limit:
                return out
            continue
        for x in avail:
            new_indeg = list(indeg_t)
            new_avail = list(avail)
            new_avail.remove(x)
            for w in adj[x]:
                new_indeg[w]-=1
                if new_indeg[w]==0:
                    new_avail.append(w)
            stack.append((pref+(x,), tuple(sorted(new_avail)), tuple(new_indeg)))
    return out

def L_filter(N, edges):
    """Apply L = EM ∘ NC ∘ ID to a candidate relation set.
    Returns (status, detail):
      - ('reject:cycle', None) if cyclic (violates NC)
      - ('partial', None) if acyclic but multiple linear extensions (EM not forced)
      - ('total', pi) if a unique total order exists (EM selects it)
    """
    if not is_acyclic(N, edges):
        return ('reject:cycle', None)
    exts = linear_extensions(N, edges, limit=2)
    if len(exts)==1:
        return ('total', exts[0])
    else:
        return ('partial', None)

def inversion_count(perm):
    s = 0
    for i in range(len(perm)):
        for j in range(i+1,len(perm)):
            if perm[i]>perm[j]:
                s+=1
    return s

## 3) Toy Examples
We illustrate the three outcomes of $L$ on small graphs.

In [ ]:
# N=3, edges forming a cycle (reject)
N=3; edges_cycle=[(1,2),(2,3),(3,1)]
L_filter(N, edges_cycle)

In [ ]:
# N=3, partial order (multiple linear extensions)
N=3; edges_partial=[(1,2)]
L_filter(N, edges_partial), linear_extensions(N, edges_partial)[:5]

In [ ]:
# N=4, total (unique linear extension)
N=4; edges_total=[(1,2),(2,3),(3,4)]
L_filter(N, edges_total)

## 4) Feasibility Ratio and the N=4 Threshold (reprise)
Define the **consistency/feasibility ratio**
$$\rho_N = \frac{N!}{2^{\,N(N-1)/2}},$$
the fraction of all pairwise sign choices that correspond to a **consistent** total order. We plot $\rho_N$ and show its steep decline with $N$, highlighting the special role of $N=4$ discussed in `00d_Origin_of_N`.

In [ ]:
import math
import matplotlib.pyplot as plt

def rho_N(N):
    return math.factorial(N) / (2**(N*(N-1)//2))

Ns = list(range(2,11))
vals = [rho_N(n) for n in Ns]

plt.figure(figsize=(6,4))
plt.plot(Ns, vals, marker='o')
plt.yscale('log')
plt.xlabel('N')
plt.ylabel('rho_N (log scale)')
plt.title('Feasibility ratio rho_N vs N (log scale)')
plt.tight_layout()
plt.savefig('/mnt/data/rhoN_plot.png', dpi=160)
plt.show()
vals

## 5) Sampling Posets and Linear Extensions
We sample random directed graphs (edge probability $p$), filter them with $L$, and estimate how often we get rejection / partial / unique total. This connects the **structure of $I$** (random raw distinctions) to observable output types after filtering.

In [ ]:
def random_edges(N, p=0.3, rng=None):
    if rng is None:
        rng = random
    E=[]
    for i in range(1,N+1):
        for j in range(1,N+1):
            if i==j: continue
            if rng.random()<p:
                E.append((i,j))
    return E

def trial_stats(N=5, T=200, p=0.3, seed=0):
    rng = random.Random(seed)
    counts={'reject:cycle':0,'partial':0,'total':0}
    for _ in range(T):
        E = random_edges(N,p,rng)
        status, pi = L_filter(N,E)
        counts[status]+=1
    return counts

stats_5 = trial_stats(N=5, T=400, p=0.25, seed=1)
stats_6 = trial_stats(N=6, T=400, p=0.2, seed=2)
stats_5, stats_6

In [ ]:
labels = ['reject:cycle','partial','total']
y5 = [stats_5[k] for k in labels]
y6 = [stats_6[k] for k in labels]

import matplotlib.pyplot as plt
x = np.arange(len(labels))
w = 0.35
plt.figure(figsize=(6,4))
plt.bar(x-w/2, y5, width=w, label='N=5')
plt.bar(x+w/2, y6, width=w, label='N=6')
plt.xticks(x, labels, rotation=15)
plt.ylabel('count (T trials)')
plt.title('Outcomes after L-filter on random raw distinctions')
plt.legend()
plt.tight_layout()
plt.savefig('/mnt/data/L_filter_outcomes.png', dpi=160)
plt.show()

## 6) Bridge Hooks to Later Notebooks
- **To 02 Geometry:** total orders (permutations) realize $S_N$; adjacent swaps → Coxeter generators of $A_{N-1}$.
- **To 03 Dynamics & Time:** inversion count on total orders is a Lyapunov functional for refinement (time emerges as descent).  
- **To 04 Qudit Bridge:** probability simplex maps affinely to the sum‑zero subspace $V$; $S_N$ orbits ↔ permutohedra.
- **To 06 Spacetime Test:** global flow alignment selects a time axis with 3 orthogonal spatial axes in 4D.

This notebook provides the **ontological substrate** (space of partial orders) on which all subsequent constructions operate.

## 7) Takeaways
- **$I$ as potential distinctions:** modelled minimally by finite digraphs/partial orders on a countable carrier.
- **$L$ as feasibility + completion:** Non‑Contradiction (acyclicity) and Excluded Middle (linear extension) yield actualities as **total orders** when feasible.
- **Predictive handle:** The shrinking feasibility ratio $\rho_N$ and random‑poset outcomes give quantitative levers that connect the ontology to emergent structure (e.g., the special role of $N=4$).

## 8) Artifact Manifest
- Figure: `/mnt/data/rhoN_plot.png`  
- Figure: `/mnt/data/L_filter_outcomes.png`

These can be ingested by `07_Figure_Bundle.ipynb` with stable filenames.